In [1]:
%cd ..

/Users/juliuslipp/projects/current/mixedbread/mixedbread-ai/packages/ofen


/Users/juliuslipp/projects/current/mixedbread/mixedbread-ai/packages/ofen/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
from ofen.configs import DeviceConfig, OptimizationConfig
from ofen.models import TextEncoderConfig, TextEncoder

model_names = [
    "mixedbread-ai/mxbai-embed-large-v1",
    "mixedbread-ai/mxbai-embed-2d-large-v1",
    "sentence-transformers/all-MiniLM-L6-v2",
    "WhereIsAI/UAE-Large-V1",
    "intfloat/e5-large-v2",
    "BAAI/bge-large-en-v1.5",
    "thenlper/gte-large",
    "thenlper/gte-large-zh",
    "intfloat/multilingual-e5-large",
    "intfloat/multilingual-e5-base"
]

optimization_configs = [
    OptimizationConfig(quantization="int8")
]

device_config = DeviceConfig("cpu")
model_cls = TextEncoder
model_config_cls = TextEncoderConfig

In [2]:
from ofen.utilities.benchmark_tools import BenchmarkTools

results = {}
for model_name in model_names:
    text_encoder_config = model_config_cls.from_pretrained(model_name)
    base_model = model_cls(
        model_name,
        text_encoder_config=text_encoder_config,
        device_config=device_config
    )

    models = [base_model]
    for i, cfg in enumerate(optimization_configs):
        models.append(model_cls(
            model_name,
            text_encoder_config=text_encoder_config,
            optimization_config=cfg,
            device_config=device_config,
        ))

    comparison_result = BenchmarkTools.compare_models(models)
    results[model_name] = comparison_result

/Users/juliuslipp/projects/current/mixedbread/mixedbread-ai/packages/ofen/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
/Users/juliuslipp/projects/current/mixedbread/mixedbread-ai/packages/ofen/.venv/lib/python3.11/site-packages/optimum/bettertransformer/models/encoder_models.py:301: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorc

RuntimeError: The expanded size of the tensor (128) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [64, 128, 384].  Tensor sizes: [64, 512, 1]

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

# Data input
data = results

# Prepare data for visualization
def prepare_data_for_visualization(data):
    records = []
    for model, results in data.items():
        for idx, result in enumerate(results):
            record = {
                'model': model,
                'comparison': idx,
                'mean_diff': result.mean_diff,
                'max_diff': result.max_diff,
                'min_diff': result.min_diff,
                'avg_time': result.avg_time,
                'total_time': result.total_time,
                'is_within_range': result.is_within_range
            }
            records.append(record)
    return pd.DataFrame(records)

df = prepare_data_for_visualization(data)

# Plotting the data
def plot_comparison_results(df):
    models = df['model'].unique()
    n_models = len(models)
    fig, axes = plt.subplots(n_models, 3, figsize=(18, 6 * n_models))

    if n_models == 1:
        axes = [axes]  # To make it iterable

    for i, model in enumerate(models):
        model_df = df[df['model'] == model]

        # Plot mean, max, and min differences
        for col, metric in enumerate(['mean_diff', 'max_diff', 'min_diff']):
            axes[i][0].plot(
                model_df['comparison'], model_df[metric], label=metric.replace('_', ' ').title(),
                marker='o', linestyle='-', color='b' if metric == 'mean_diff' else 'g' if metric == 'max_diff' else 'r'
            )
        for index, row in model_df.iterrows():
            if not row['is_within_range']:
                axes[i][0].axvline(x=row['comparison'], color='k', linestyle='--')
        axes[i][0].set_title(f'{model} - Mean, Max, and Min Differences')
        axes[i][0].legend()

        # Plot average time
        axes[i][1].plot(model_df['comparison'], model_df['avg_time'], label='Avg Time', marker='o', linestyle='-', color='g')
        for index, row in model_df.iterrows():
            if not row['is_within_range']:
                axes[i][1].axvline(x=row['comparison'], color='k', linestyle='--')
        axes[i][1].set_title(f'{model} - Average Time')
        axes[i][1].legend()

        # Plot total time
        axes[i][2].plot(model_df['comparison'], model_df['total_time'], label='Total Time', marker='o', linestyle='-', color='r')
        for index, row in model_df.iterrows():
            if not row['is_within_range']:
                axes[i][2].axvline(x=row['comparison'], color='k', linestyle='--')
        axes[i][2].set_title(f'{model} - Total Time')
        axes[i][2].legend()

    plt.tight_layout()
    plt.show()

plot_comparison_results(df)


KeyboardInterrupt

